In [1]:
import numpy as np  
import numpy.polynomial.legendre as npl
import sympy as sp
from scipy.special import legendre

In [2]:
class Cuadrature:
    def __init__(self,num_points,nodes=None,weight=None):
        self.num_points=num_points
    def getNodes(self):
        return self.nodes
    def getWeight(self):
        return self.weight
    def Integrate(self,f):
        return self.Integrate_List(f(np.array(self.nodes)))
    def Integrate_List(self,lista):
        if (len(lista)==len(self.weight)):
            return np.array(lista).dot(np.array(self.weight))
        else:
            raise Exception("No has metido correctamente la lista")

In [3]:
class Legendre(Cuadrature):
    def __init__(self,num_points,nodes=None,weight=None):
        Cuadrature.__init__(self,num_points)
        self.nodes=npl.leggauss(self.num_points)[0]  
        self.weight=npl.leggauss(self.num_points)[1]  
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [4]:
class Trapecio(Cuadrature):
    def __init__(self,num_points,nodes=None,weight=None):
        Cuadrature.__init__(self,num_points)
        self.nodes=[*np.arange(-1,1,float(2/self.num_points)).tolist(),1]
        self.weight=[2/(2*self.num_points),*[2/self.num_points]*(self.num_points-1),2/(2*self.num_points)]
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [5]:
class Lobato(Cuadrature):
    def __init__(self, num_points,nodes=None,weight=None):
        Cuadrature.__init__(self, num_points)
        self.nodes=[-1,*sorted(list(legendre(self.num_points).deriv().r)),1]
        self.weight=2/(self.num_points*(self.num_points+1)*(legendre(self.num_points)(self.nodes))**2)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [6]:
print(Legendre(3))

Nodos: [-0.77459667  0.          0.77459667] 
Pesos: [0.55555556 0.88888889 0.55555556] 


In [7]:
print(Trapecio(5))

Nodos: [-1.0, -0.6, -0.19999999999999996, 0.20000000000000018, 0.6000000000000001, 1] 
Pesos: [0.2, 0.4, 0.4, 0.4, 0.4, 0.2] 


In [8]:
print(Lobato(4))

Nodos: [-1, -0.6546536707079773, 3.23815048849004e-17, 0.6546536707079773, 1] 
Pesos: [0.1        0.54444444 0.71111111 0.54444444 0.1       ] 


Veamos que integra correctamente:

In [10]:
x = sp.Symbol('x')
y=x**10
sp.integrate(y,(x,-1,1))

2/11

In [11]:
f= lambda x: x**10

Legendre(3).Integrate(f)

0.08640000000000005

In [12]:
Legendre(10).Integrate(f)

0.18181818181818166

In [13]:
Trapecio(3).Integrate(f)

0.6666892467837445

In [14]:
Trapecio(10).Integrate(f)

0.2454103039999999

In [15]:
Lobato(3).Integrate(f)

0.33386666666666664

In [16]:
Lobato(10).Integrate(f)

0.1818181818181771

# Pasos que he seguido a la hora de construir las clases

## Legendre:
Los nodos y pesos de Legendre se obtienen directamente con la función

In [17]:
npl.leggauss(3)

(array([-0.77459667,  0.        ,  0.77459667]),
 array([0.55555556, 0.88888889, 0.55555556]))

## Lobato:
En la fórmula de Gauss_Lobato los nodos interiores son los ceros de la derivada del polinomio de Legendre $L_N'(x_i)=0$ y los pesos los he sacado de la fórmula del libro de Quarteroni.

In [18]:
n=2 #son n+1 puntos en total del x_0 al x_n

interior=legendre(n).deriv().r
nodos=[-1,*sorted(list(interior)),1]

pesos=2/(n*(n+1)*(legendre(n)(nodos))**2)

print(nodos,pesos)

[-1, 0.0, 1] [0.33333333 1.33333333 0.33333333]


## Trapecios:
En la fórmula de los trapecios he considerado el intervalo $(-1,1)$ en $n$ intervalos, de modo que tengo $n+1$ puntos, el incremento y por tanto h es, $h=\frac{b-a}n$ y los pesos en los extremos son $\frac h 2$ y en los nodos interiores $h$.

In [19]:
a,b=-1,1
n=6
[*np.arange(-1,1,float((b-a)/n)).tolist(),1] #nodos en Trapecios

[-1.0,
 -0.6666666666666667,
 -0.3333333333333335,
 -2.220446049250313e-16,
 0.33333333333333304,
 0.6666666666666663,
 1]

In [20]:
n=6 #x_0,...x_6 7 puntos
[2/(2*n),*[2/n]*(n-1),2/(2*n)] #pesos en Trapecios

[0.16666666666666666,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.16666666666666666]

## Con Dani y Rafa Lunes 14 Marzo

In [21]:
class Cuadrature:
    def __init__(self,nodes=None,weight=None,num_points=None):
        if (nodes!=None and weight!=None):
            self.nodes = nodes
            self.weight = weight
            print(self.nodes,self.weight)
        elif(num_points!=None):
            self.nodes, self.weight= npl.leggauss(num_points)
            print(self.nodes,self.weight)
        else:
            raise Exception("No has metido correctamente los parámetros en la clase")
    def getNodes(self):
        return self.nodes
    def getWeight(self):
        return self.weight
    def Integrate(self,f):
        return self.Integrate_List(f(np.array(self.nodes)))
    def Integrate_List(self,lista):
        if (len(lista)==len(self.weight)):
            return np.array(lista).dot(np.array(self.weight))
        else:
            raise Exception("No has metido correctamente la lista")
        

In [22]:
Cuadrature(4)

Exception: No has metido correctamente los parámetros en la clase

In [25]:
cuad_gauss=Cuadrature(num_points=6)

[-0.93246951 -0.66120939 -0.23861919  0.23861919  0.66120939  0.93246951] [0.17132449 0.36076157 0.46791393 0.46791393 0.36076157 0.17132449]


In [26]:
f= lambda x: x**10
cuad_gauss.Integrate(f)

0.18181818181818124

In [27]:
lista=[2,4,5,0]
cuad_gauss.Integrate_List(lista)

Exception: No has metido correctamente la lista